# Evan Trock Final Data Analysis Project

## Freddie Mac Fannie Mae Data

In [1]:
import pandas as pd
import statsmodels.api as sm
import numpy as np
from statsmodels.formula.api import ols as sm_ols
from statsmodels.iolib.summary2 import summary_col
# Load the Excel file
df = pd.read_csv('Spring2025_Freddie_Assignment_Data.csv')

missing_values = [9, 99, 999, 9999]
df.replace(missing_values, np.nan, inplace=True)

df.head()

,oyear,Credit Score,First Payment Date,First Time Hombebuyer Flag,Maturity Date,MSA,MI%,#Units,Occupancy,CLTV,...,Number Borrowers,Seller Name,Servicer Name,Super Conforming Flag,Pre-Relief Refinance Loan Sequence Number,ProgramIndicator,RELIEF REFINANCE INDICATOR,PROPERTY VALUATION METHOD,INTEREST ONLY INDICATOR,MI CANCELLATION INDICATOR
0,1999,689.0,199909,N,202908,12060.0,0.0,1.0,P,70.0,...,1.0,"NORWEST MORTGAGE, INC.","WELLS FARGO HOME MORTGAGE, INC.",NaN,NaN,NaN,NaN,NaN,N,NaN
1,1999,641.0,199906,N,202905,49340.0,30.0,1.0,P,95.0,...,2.0,"COUNTRYWIDE HOME LOANS, INC.","COUNTRYWIDE HOME LOANS, INC.",NaN,NaN,NaN,NaN,NaN,N,NaN
2,1999,743.0,199912,N,202911,24580.0,25.0,1.0,P,89.0,...,1.0,"NORWEST MORTGAGE, INC.","WELLS FARGO HOME MORTGAGE, INC.",NaN,NaN,NaN,NaN,NaN,N,NaN
3,1999,679.0,200002,N,203001,35004.0,0.0,1.0,P,67.0,...,2.0,"ABN AMRO MORTGAGE GROUP, INC.","ABN AMRO MORTGAGE GROUP, INC.",NaN,NaN,NaN,NaN,NaN,N,NaN
4,1999,741.0,199908,Y,202907,10580.0,0.0,1.0,P,70.0,...,2.0,"NATIONSBANK, N.A.,DBA BANK OF AMERICA MORTGAGE","BANK OF AMERICA, N.A.",NaN,NaN,NaN,NaN,NaN,N,NaN


In [2]:
#Preprocess the data
# Rename columns based on Origination Data
rename_columns = {
    'Orig. Int Rate': 'Rate',
    'UPB': 'UPB',
    'Loan Term': 'Term',
    'LTV': 'LTV',
    'DTI': 'DTI',
    'Credit Score': 'FICO',
    'Number Borrowers': 'Borrowers',
    'First Time Hombebuyer Flag': 'First',
    'PPM ': 'Penalty',
    'Loan Purpose': 'Purpose',
    'oyear': 'Year',
    'Seller Name': 'Seller'
}
df_clean = df.rename(columns=rename_columns)

# Ensure correct data types
df_clean['Rate'] = pd.to_numeric(df_clean['Rate'], errors='coerce')
df_clean['UPB'] = pd.to_numeric(df_clean['UPB'], errors='coerce')
df_clean['Term'] = pd.to_numeric(df_clean['Term'], errors='coerce')
df_clean['LTV'] = pd.to_numeric(df_clean['LTV'], errors='coerce')
df_clean['DTI'] = pd.to_numeric(df_clean['DTI'], errors='coerce')
df_clean['FICO'] = pd.to_numeric(df_clean['FICO'], errors='coerce')
df_clean['Borrowers'] = pd.to_numeric(df_clean['Borrowers'], errors='coerce')

# --- Fix Categorical Variables ---
for var in ['First', 'Penalty']:
    df_clean[var] = df_clean[var].apply(lambda x: 1 if str(x).strip().upper() == 'Y' else (0 if str(x).strip().upper() == 'N' else np.nan))

# --- Correct Purchase Indicator (final fix for Freddie codes) ---
df_clean['Purchase'] = df_clean['Purpose'].apply(
    lambda x: 1 if str(x).strip().upper() == 'P' else 
              (0 if str(x).strip().upper() in ['R', 'C'] else np.nan)
)

# Create additional indicators
df_clean['< 680'] = df_clean['FICO'].apply(lambda x: 1 if pd.notnull(x) and x < 680 else 0)
df_clean['Single'] = df_clean['Borrowers'].apply(lambda x: 1 if x == 1 else 0)


In [3]:
# Rename columns
cols_stats = ['Rate', 'UPB', 'Term', 'LTV', 'DTI', 'FICO', 'Borrowers', 'First', 'Penalty']
df_stats = df_clean[cols_stats].copy()

# Rounding setup
rounding = {'Rate': 1, 'UPB': 0, 'Term': 0, 'LTV': 1, 'DTI': 1, 'FICO': 0, 'Borrowers': 1, 'First': 2, 'Penalty': 3}

# Summary statistics
summary = {}
for col in df_stats.columns:
    stats = {
        'Mean': round(df_stats[col].mean(skipna=True), rounding[col]),
        'Median': round(df_stats[col].median(skipna=True), rounding[col]),
        'Std Dev': round(df_stats[col].std(skipna=True), rounding[col]),
        'Min': round(df_stats[col].min(skipna=True), rounding[col]),
        'Max': round(df_stats[col].max(skipna=True), rounding[col]),
        'Count': int(df_stats[col].count())
    }
    summary[col] = stats
summary_df = pd.DataFrame(summary).T

print("\n--- Part 1: Summary Statistics ---")
print(summary_df)


--- Part 1: Summary Statistics ---
                 Mean    Median     Std Dev      Min        Max    Count
Rate            5.200       5.0       1.400      1.8       12.8  75701.0
UPB        207109.000  178000.0  125261.000  10000.0  1564000.0  75750.0
Term          315.000     360.0      76.000     60.0      480.0  75750.0
LTV            71.800      75.0      18.900      6.0      386.0  75676.0
DTI            34.100      35.0      10.800      1.0       65.0  69809.0
FICO          741.000     751.0      52.000    300.0      842.0  75610.0
Borrowers       1.600       2.0       0.500      1.0        4.0  75736.0
First           0.140       0.0       0.350      0.0        1.0  75709.0
Penalty         0.001       0.0       0.036      0.0        1.0  75750.0


In [4]:
# Calculate the Pearson correlation matrix
correlation_matrix = df_stats.corr(method='pearson').round(2)

print("\n--- Part 2: Correlation Matrix ---")
print(correlation_matrix)



--- Part 2: Correlation Matrix ---
           Rate   UPB  Term   LTV   DTI  FICO  Borrowers  First  Penalty
Rate       1.00 -0.25  0.19  0.05  0.07 -0.28       0.04   0.03     0.04
UPB       -0.25  1.00  0.22  0.13  0.13  0.14       0.08   0.09    -0.03
Term       0.19  0.22  1.00  0.28  0.17 -0.05      -0.08   0.18    -0.04
LTV        0.05  0.13  0.28  1.00  0.14 -0.13      -0.06   0.26    -0.02
DTI        0.07  0.13  0.17  0.14  1.00 -0.16      -0.10   0.05    -0.02
FICO      -0.28  0.14 -0.05 -0.13 -0.16  1.00      -0.04  -0.01    -0.01
Borrowers  0.04  0.08 -0.08 -0.06 -0.10 -0.04       1.00  -0.09    -0.01
First      0.03  0.09  0.18  0.26  0.05 -0.01      -0.09   1.00    -0.01
Penalty    0.04 -0.03 -0.04 -0.02 -0.02 -0.01      -0.01  -0.01     1.00


In [5]:
cols_avg = ['Year', 'Rate', 'Purchase', 'UPB', 'LTV', 'DTI', 'FICO', '< 680', 'Single']

# Only drop rows if Year or Rate are missing
df_avg = df_clean[cols_avg].dropna(subset=['Year', 'Rate'])

# Group by year and calculate mean, allowing missing in non-key columns
averages_by_year = df_avg.groupby('Year').mean()

# Overall averages across all years
overall_avg = pd.DataFrame(df_avg.drop(columns=['Year']).mean(axis=0)).T
overall_avg.index = ['Overall Averages']

overall_avg.index = ['Overall Averages']

averages_with_totals = pd.concat([averages_by_year, overall_avg])

print("\n--- Part 3: Yearly Averages (plus Overall) ---")
averages_with_totals.round(2)



--- Part 3: Yearly Averages (plus Overall) ---


,Rate,Purchase,UPB,LTV,DTI,FICO,< 680,Single
1999,7.33,0.68,115237.47,71.36,31.86,714.30,0.26,0.35
2000,8.12,0.80,123532.61,72.59,33.48,715.63,0.26,0.39
2001,6.86,0.49,138196.13,70.43,32.42,720.02,0.23,0.34
2002,6.40,0.46,143826.33,68.32,32.10,723.82,0.21,0.34
2003,5.53,0.41,141283.00,66.31,31.13,729.86,0.18,0.33
2004,5.67,0.56,157035.33,68.75,33.71,722.70,0.23,0.39
2005,5.81,0.50,170089.00,69.46,35.35,722.62,0.23,0.42
2006,6.41,0.55,179737.33,70.08,36.12,722.55,0.24,0.45
2007,6.37,0.57,181224.00,71.39,36.78,726.42,0.22,0.46
2008,6.06,0.56,202259.33,70.00,36.43,740.94,0.14,0.46


In [6]:
seller_stats = df_clean['Seller'].value_counts().reset_index()
seller_stats.columns = ['Seller', 'Frequency']

# Percent and cumulative percent
total_loans = seller_stats['Frequency'].sum()
seller_stats['Percent'] = (seller_stats['Frequency'] / total_loans * 100).round(2)
seller_stats['Cumulative'] = seller_stats['Percent'].cumsum().round(2)
seller_stats['Number'] = range(1, len(seller_stats) + 1)

seller_stats = seller_stats[['Number', 'Seller', 'Frequency', 'Percent', 'Cumulative']]

print("\n--- Part 4: Seller Frequency Table ---")
print(seller_stats)



--- Part 4: Seller Frequency Table ---
     Number                           Seller  Frequency  Percent  Cumulative
0         1                    Other sellers      20085    26.51       26.51
1         2           WELLS FARGO BANK, N.A.      10167    13.42       39.93
2         3  WELLS FARGO HOME MORTGAGE, INC.       3659     4.83       44.76
3         4            BANK OF AMERICA, N.A.       3554     4.69       49.45
4         5                   U.S. BANK N.A.       3160     4.17       53.62
..      ...                              ...        ...      ...         ...
104     105           PROSPECT MORTGAGE, LLC          8     0.01       99.93
105     106                        ALLY BANK          7     0.01       99.94
106     107                 NORTHPOINTE BANK          5     0.01       99.95
107     108     PACIFIC UNION FINANCIAL, LLC          3     0.00       99.95
108     109               PULTE MORTGAGE LLC          3     0.00       99.95

[109 rows x 5 columns]


In [7]:
# --- PART 5: Regression Analysis ---
df_reg = df_clean[(df_clean['Year'] >= 2013) & (df_clean['Year'] <= 2024)]

# Prepare regression dataset
df_reg = df_reg[['Rate', 'UPB', 'Term', 'LTV', 'DTI', 'FICO', 'Year']].dropna()

# Create year dummies
year_dummies = pd.get_dummies(df_reg['Year'].astype(int), prefix='Year')
year_dummies.drop('Year_2013', axis=1, inplace=True, errors='ignore')

# Define X and y
X = pd.concat([df_reg[['UPB', 'Term', 'LTV', 'DTI', 'FICO']], year_dummies], axis=1)
y = df_reg['Rate']

# Add constant
X = sm.add_constant(X)
X = X.astype(float)
y = y.astype(float)
#Fit regression model
model = sm.OLS(y, X)
results = model.fit()

print("\n--- Part 5: Regression Results ---")
print(results.summary())



--- Part 5: Regression Results ---
                            OLS Regression Results                            
Dep. Variable:                   Rate   R-squared:                       0.801
Model:                            OLS   Adj. R-squared:                  0.801
Method:                 Least Squares   F-statistic:                     8017.
Date:                Mon, 05 May 2025   Prob (F-statistic):               0.00
Time:                        14:33:28   Log-Likelihood:                -25745.
No. Observations:               31815   AIC:                         5.152e+04
Df Residuals:                   31798   BIC:                         5.167e+04
Df Model:                          16                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          4